# Miniproject 1
Fingermovements predicting

**///////////// TODO //////////////**
- convnet
- LSTM from slides
- gru from slides
- covnet wit dilution like class


** /////////// immidiate todo ////////////////**
- continue with LSTM
    - maybe need to give hidden in every iteration.
    - make new train(function) for lstm
    - 1,50,1 is a problem how to give one class? (maybe last linearlayer?)

In [155]:
# IMPORTS
import torch
from torch import Tensor, nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
import numpy as np
from math import log10
import dlc_bci as bci
import torch.nn.functional as F

# Our own code
import helpers as HL
import models as ML

%reload_ext autoreload
%autoreload 2

### Importing data

In [156]:
train_data, test_data = HL.import_data(flatten=False)

Original data format: 
<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])
Modified train_data.data_tensor shape:  torch.Size([316, 28, 50])
Modified train_data.target_tensor shape:  torch.Size([316, 2])


### Create new models if you want

In [157]:

def SenseNet(input_dim, output_dim, nb_hidden_layers=1, hidden_width=100, dropout_rate=False):
    """
    Fully connected feedforward neural net with adjustible, but uniform width of hidden layers and and adjustable depth
    """
    if nb_hidden_layers < 1:
        print("you need at least one hidden layer")
        model = None
    else:
        layers = []
        layers.append(nn.Linear(input_dim, hidden_width))
        layers.append(nn.ReLU())
        for i in range(nb_hidden_layers-1):
            layers.append(nn.Linear(hidden_width, hidden_width))
            layers.append(nn.ReLU())
            if type(dropout_rate) == float and dropout_rate > 0.0 and dropout_rate < 1.0:
                layers.append(nn.Dropout(p=dropout_rate))

        layers.append(nn.Linear(hidden_width, output_dim))
        layers.append(nn.Softmax())

        model = nn.Sequential(*layers)
        return model

class BiRNN(nn.Module):
    def __init__(self, input_dim, hidden_width, nb_layers, output_dim):
        super(BiRNN, self).__init__()
        self.hidden_width = hidden_width
        self.nb_layers = nb_layers
        self.lstm = nn.LSTM(input_dim, hidden_width, nb_layers, 
                            batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_width*2, output_dim)  # 2 for bidirection 
    
    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.nb_layers*2, x.size(0), self.hidden_width)) # 2 for bidirection 
        c0 = Variable(torch.zeros(self.nb_layers*2, x.size(0), self.hidden_width))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out
    
    
    
    


    
    


class conv_net1(nn.Module):
    def __init__(self, input_dim, hidden_width, nb_layers, output_dim):
        super().__init__()
        # makes 224 features out of the time series and 24 features
        # feature making
        self.conv1 = nn.Conv1d(input_dim, 56, kernel_size=4, stride=4, padding=0)
        self.conv2 = nn.Conv1d(56, 112, kernel_size=3, stride=3, padding=0)
        self.conv3 = nn.Conv1d(112, 224, kernel_size=3, stride=1, padding=0)
        self.conv4 = nn.Conv1d(224, 224, kernel_size=2, stride=1, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        #self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc1 = torch.nn.Linear(224, hidden_width)
        self.r = nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_width, output_dim)
        self.smax = torch.nn.Softmax(dim=0)
    
    def forward(self, x):
        # feature making
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        
        # remove redundant dimension before classification
        out = out.view(out.shape[0],-1)
        
        # classification
        out = self.fc1(out)
        out = self.r(out)
        out = self.fc2(out)
        out = self.smax(out)
        return out
    
class conv_net2(nn.Module):
    def __init__(self, input_dim, hidden_width, nb_layers, output_dim):
        super().__init__()
        # makes 224 features out of the time series and 24 features
        # feature making
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(input_dim, 56, kernel_size=4, stride=4, padding=0),
            nn.BatchNorm1d(56),
            nn.ReLU())
            #,nn.MaxPool1d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv1d(56, 112, kernel_size=3, stride=3, padding=0),
            nn.BatchNorm1d(112),
            nn.ReLU())
            #nn.MaxPool1d(2))
        
        self.layer3 = nn.Sequential(
            nn.Conv1d(112, 224, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm1d(224),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv1d(224, 224, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm1d(224),
            nn.ReLU())
        
        
        
        #4608 input features, 64 output features (see sizing flow below)
        #self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.classification = nn.Sequential(
            nn.Linear(224, hidden_width),
            nn.ReLU(),
            nn.Linear(hidden_width, output_dim),
            nn.Softmax(dim=0))
    
    def forward(self, x):
        # feature making
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        # remove redundant dimension before classification
        out = out.view(out.shape[0],-1)
        
        # classification
        out = self.classification(out)
        return out

## TODO
# - the batches fucked it up, fix it in the input.narrow() with s an t properly

class rec_net(nn.Module):
    def __init__(self, input_dim, output_dim, rec_dim, internal_dim):
        super().__init__()
        self.fc_x2v = nn.Linear(input_dim, internal_dim) # train weights of how x(input) influence v(internal state)
        self.fc_h2v = nn.Linear(rec_dim, internal_dim, bias= False) # why false bias here? train how (recurrent state) influence v(internal state)
        self.fc_v2h = nn.Linear(internal_dim, rec_dim) # train how v(internal state) influence h(recurrent state)
        self.fc_h2y = nn.Linear(rec_dim, output_dim) # train how to predict from h(recurrent state)
        
    def forward(self, input):
        # initialize recurrent state with zeros in shape (1,output_dim)
        h = Variable(input.data.new(1, self.fc_h2y.weight.size(1)).zero_())
        
        # loop through every sample in batch
        for s in range(input.size(0)):
            # initialize recurrent state with zeros in shape (1,output_dim)
            h = Variable(input.data.new(1, self.fc_h2y.weight.size(1)).zero_())
            
            # loop through every part in sequence (first dim is "time")
            for t in range(input.size(2)):
                # t = data in that part of sequence,
                # v = internal/hidden state, 
                # h = recurrent state
                print("fc_x2v: ", self.fc_x2v)
                print("h: ", h.shape)
                print("self.fc_h2v(h): ", self.fc_h2v(h).shape)
                print("input.narrow(0, t, 1): ", input.shape)
                v = F.relu(self.fc_x2v(input[s].narrow(0, t, 1)) + self.fc_h2v(h))
                print("v: ", v)
                print("fc_v2h: ", self.fc_v2h)
                h = self.fc_v2h(v)
        # when to return???
        # gotta store vector of predictions
        return self.fc_h2y(h)
    

class recurrent_net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        
        self.hidden = self.init_hidden()
    
    def forward(self, input):  # ,hidden)
        print("input: ", input[0,:,0].unsqueeze(dim=0).shape)
        print("hidden: ", self.hidden.shape)
        combined = torch.cat((input[0,:,0].unsqueeze(dim=0), self.hidden), 1)
        self.hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output

    def init_hidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

In [158]:
################################# TESTING ##############################
drit = nn.Conv1d(28, 56, 3, stride=1, padding=0)
drit2 = torch.nn.Conv1d(56, 112, kernel_size=3, stride=1, padding=0)
drit3 = nn.Conv1d(112, 224, kernel_size=3, stride=1, padding=0)
drit4 = nn.Conv1d(224, 224, kernel_size=2, stride=1, padding=0)
pul = nn.MaxPool1d(2)

fack = Variable(torch.randn(20, 28, 50))
print("input:", fack.shape)
output = drit(fack)
print(output.shape)
output = pul(output)
print(output.shape)

output = drit2(output)
print(output.shape)
output = pul(output)
print(output.shape)

output = drit3(output)
print(output.shape)
output = pul(output)
print(output.shape)

output = drit4(output)
print(output.shape)
output = pul(output)
print(output.shape)
#output = pul(output)
#print(output.shape)


output.view(output.shape[0],-1).shape



input: torch.Size([20, 28, 50])
torch.Size([20, 56, 48])
torch.Size([20, 56, 24])
torch.Size([20, 112, 22])
torch.Size([20, 112, 11])
torch.Size([20, 224, 9])
torch.Size([20, 224, 4])
torch.Size([20, 224, 3])
torch.Size([20, 224, 1])


torch.Size([20, 224])

In [202]:
class LSTM_net(nn.Module):
    def __init__(self, dim_input, dim_recurrent, num_layers, dim_output):
        super().__init__ ()
        self.lstm = nn.LSTM(input_size = dim_input, hidden_size = dim_recurrent, num_layers = num_layers, batch_first=True)
        self.fc_o2y = nn.Linear(dim_recurrent, dim_output)
        
    def forward (self, input):
        # Makes this a batch of size 1
        # The first index is the time , sequence number is the second
        #print("input before: ", input.shape)
        input.transpose_(1,2)
        #input = input.unsqueeze(1)
        #print("input after: ", input.shape)
        # Get the activations of all layers at the last time step
        output, _ = self.lstm(input)
        # Drop the batch index
        output = output.squeeze(1)
        output = output.narrow(0, output.size(0)-1, 1)
        prev_output = self.fc_o2y(F.relu(output))
        #print("prev_output", prev_output)
        return F.softmax(prev_output, dim=2) # F.relu substituted with softmax
    

### Choose model and define loss criterion and train

In [203]:
input_dim = len(train_data[0][0])
output_dim = train_data.target_tensor.size(1)
nb_hidden_layers = 3
hidden_width = 500
dropout_rate=0.4

#input_dim=50 # for LSTM input dim might be the timedim

print("input_dim: ", input_dim)
print("output_dim: ", output_dim)

# initialize the model
densemodel_1 = ML.DenseNet(input_dim, output_dim, nb_hidden_layers, hidden_width, dropout_rate)
linear_model = ML.Linear_regression_model(input_dim, output_dim)

sensemodel = SenseNet(input_dim, output_dim, nb_hidden_layers, hidden_width, dropout_rate)
birnn = BiRNN(input_dim, hidden_width, nb_hidden_layers, output_dim)
conv_model1 = conv_net1(input_dim, hidden_width, nb_hidden_layers, output_dim)
conv_model2 = conv_net2(input_dim, hidden_width, nb_hidden_layers, output_dim)
rec_model = rec_net(input_dim, output_dim, 77, 77) # input_dim, output_dim, rec_dim, internal_dim)
rnn_model = recurrent_net(input_dim, hidden_width, output_dim)
lstm_model = LSTM_net(input_dim, dim_recurrent=111, num_layers=1, dim_output=output_dim)

model = lstm_model

# define loss criterion
criterion = nn.BCELoss()

print(model)


input_dim:  28
output_dim:  2
LSTM_net(
  (lstm): LSTM(28, 111, batch_first=True)
  (fc_o2y): Linear(in_features=111, out_features=2, bias=True)
)


In [221]:
tbone = Tensor([[1,2,3],
                [4,5,6]])
print(tbone)
tbone.topk(1) # return values in dim=? and index of max


 1  2  3
 4  5  6
[torch.FloatTensor of size 2x3]



(
  3
  6
 [torch.FloatTensor of size 2x1], 
  2
  2
 [torch.LongTensor of size 2x1])

### Training model

In [226]:
trained_model = HL.train_model(train_data, test_data, model, criterion, learning_rate=0.0002, epochs=20, batch_size=1, checkpoint_name='testingtesting')

Number of trainsamples:  316
Number of testsamples:  100
===> Epoch 0 Complete: Avg. Loss: 0.1593
===> Prediction TRAIN-error: 0.0633
===> Avg. PSNR: 9.8220 dB
===> Prediction  TEST-error: 0.3600
Checkpoint: checkpoint_models/testingtesting_epoch_0.pth -------------------------------------


/Users/havardbjornoy/anaconda3/lib/python3.6/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type LSTM_net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


===> Epoch 2 Complete: Avg. Loss: 0.1548
===> Prediction TRAIN-error: 0.0570
===> Avg. PSNR: 9.8833 dB
===> Prediction  TEST-error: 0.3500
Checkpoint: checkpoint_models/testingtesting_epoch_2.pth -------------------------------------
===> Epoch 4 Complete: Avg. Loss: 0.1561
===> Prediction TRAIN-error: 0.0665
===> Avg. PSNR: 9.8574 dB
===> Prediction  TEST-error: 0.3500
Checkpoint: checkpoint_models/testingtesting_epoch_4.pth -------------------------------------
===> Epoch 6 Complete: Avg. Loss: 0.1510
===> Prediction TRAIN-error: 0.0633
===> Avg. PSNR: 9.9247 dB
===> Prediction  TEST-error: 0.3600
Checkpoint: checkpoint_models/testingtesting_epoch_6.pth -------------------------------------
===> Epoch 8 Complete: Avg. Loss: 0.1492
===> Prediction TRAIN-error: 0.0601
===> Avg. PSNR: 9.9429 dB
===> Prediction  TEST-error: 0.3600
Checkpoint: checkpoint_models/testingtesting_epoch_8.pth -------------------------------------
===> Epoch 10 Complete: Avg. Loss: 0.1482
===> Prediction TRAIN-

## Some results
conv_net1()
- trained_model = HL.train_model(train_data, test_data, model, criterion, learning_rate=0.000006, epochs=50, batch_size=8, checkpoint_name='testingtesting')
- hadde litt høyere learningrate i starten, mye flere epochs

===> Epoch 45 Complete: Avg. Loss: 0.0932
===> Prediction TRAIN-error: 0.0063
===> Avg. PSNR: -0.1346 dB
===> Prediction  TEST-error: 0.2692


In [ ]:
print(type(test_data.target_tensor))
values, indices = test_data.target_tensor.max(1)
print(values, indices)